In [1]:
import gym_examples
import gym
from stable_baselines3 import PPO
from sb3_contrib import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from gym.wrappers import FlattenObservation

In [2]:
gym.__version__

'0.26.2'

In [3]:
env = gym.make("gym_examples/StarWars-v0")

In [4]:
start_state = env.reset()
start_state

({'r2d2': array([3, 2]), 'c3po': array([2, 4]), 'vader': array([2, 1])},
 {'c3po_distance': 3.0, 'vader_distance': 2.0})

In [5]:
env.action_space

Discrete(4)

In [6]:
def masking_fn(env: gym.Env):
    return env.get_valid_action_mask()

In [7]:
env = FlattenObservation(ActionMasker(env, masking_fn))

In [10]:
# env = ActionMasker(env, masking_fn)
model = MaskablePPO(MaskableActorCriticPolicy, env, verbose=1)
model.learn(total_timesteps=100000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.7     |
|    ep_rew_mean     | -0.0037  |
| time/              |          |
|    fps             | 361      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 15          |
|    ep_rew_mean          | 0.17        |
| time/                   |             |
|    fps                  | 280         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012158364 |
|    clip_fraction        | 0.141       |
|    clip_range           | 0.2         |
|    entropy_loss   